In [234]:
import pandas as pd
import sqlalchemy
from sqlalchemy import *
import re

In [235]:
engine = create_engine("sqlite:///data/movies-db.sqlite")
conn = engine.connect()

In [14]:
df = pd.read_csv("data/genome-scores.csv")
df2 = pd.read_csv("data/genome-tags.csv")
df3 = pd.read_csv("data/links.csv")
df4 = pd.read_csv("data/movies.csv")
df5 = pd.read_csv("data/ratings.csv")
genreMatrix = pd.read_csv('clean_data/genreMatrix.csv')
links = pd.read_csv('links.csv')
other_info = pd.read_csv('clean_data/imdb_details_clean.csv')

In [101]:
my_tags = pd.read_csv('data/my_tags.csv').rename(columns={'imdbID_formatted':'imdbID'})
len(my_tags)

55909

In [102]:
ratings = df5.groupby('movieId').mean().drop(columns = ['userId', 'timestamp'])
len(ratings)

53889

In [103]:
ids = links.drop(columns=['imdbId', 'tmdbId']).rename(columns={'imdbID_formatted':'imdbID'})
len(ids)

58098

In [104]:
movies = pd.merge(ids, df4, how='inner', on='movieId')
len(movies)

58098

In [106]:
movies_ratings = pd.merge(movies, ratings, how='inner', on='movieId')
len(movies_ratings)

53889

In [107]:
movies_genres_ratings = pd.merge(movies_ratings, genreMatrix, how='inner', on='imdbID')
len(movies_genres_ratings)

53747

In [108]:
ratings_genres_tags = pd.merge(movies_genres_ratings, my_tags, how='inner', on='imdbID')
len(ratings_genres_tags)

52051

In [71]:
ratings_genres_tags.to_csv('data/movies_my_tags.csv')

In [120]:
other_info.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Season', 'Episode',
       'totalSeasons', 'seriesID'],
      dtype='object')

In [215]:
votes = []
temp = []
for i in range(0, len(other_info)):
    text = float(str(other_info['imdbVotes'][i]).replace(',',''))/2177949
    movieid = other_info['imdbID'][i]
#     year = float(str(other_info['Year'][i]).replace('Äì',''))
    mscore = other_info['Metascore'][i]
    temp = [text,movieid]
    votes.append(temp)

In [229]:
votes_score = pd.DataFrame(votes, columns =['votes','imdbID'])

In [238]:
more_info = pd.merge(movies_genres_ratings,votes_score,how='inner',on='imdbID')

In [239]:
more = pd.merge(more_info,other_info,how='inner',on='imdbID')

In [231]:
tags_df = df.pivot(index='movieId', columns='tagId', values='relevance')

In [230]:
movies_tags_ratings = pd.merge(movies_ratings, tags_df, how='inner', on='movieId')

In [243]:
more_info.to_csv('movies.csv')